# 整合

In [3]:
import requests
import googlemaps
import sqlite3
import pandas as pd
import dotenv
from Function_list import RoutePlanning

class CinemaFinder:
    def __init__(self, apikey, location):
        self.apikey = apikey
        self.gmaps = googlemaps.Client(key=self.apikey)
        self.location = location

    # 取得規劃路線距離、估計時間
    def GM(self, destination, mode='driving'):
        response = self.gmaps.distance_matrix(self.location, destination, mode=mode)
        try:
            distance = response['rows'][0]['elements'][0]['distance']['value']
            duration = response['rows'][0]['elements'][0]['duration']['text']
            return distance, duration
        except:
            return None, None
    
    # 取得附近"直線距離"前6間電影院
    def MathJudge_dist(self, df):
        df['緯度'] = df['經緯度'].map(lambda x: float(x.split(',')[0])) - float(self.location.split(',')[0])
        df['經度'] = df['經緯度'].map(lambda x: float(x.split(',')[1])) - float(self.location.split(',')[1])
        df['直線距離'] = (df['緯度'] ** 2 + df['經度'] ** 2) ** 0.5
        df = df.drop(['緯度', '經度'], axis=1)
        df = df.sort_values(['直線距離']).reset_index(drop=True)
        return df.head(6)
    
    # 取得附近"交通距離"前3間電影院
    def order_df_dist_google(self, df, mode='driving'):
        # main
        df = df.copy()
        df['temp'] = df['經緯度'].map(lambda x: self.GM(x, mode))
        df['距離(公尺)'] = df['temp'].apply(lambda x: x[0])
        df['預計時間'] = df['temp'].apply(lambda x: x[1])
        
        df.drop('temp', axis=1, inplace=True)
        df = df.sort_values('距離(公尺)').reset_index(drop=True)
        res = df[['事業名稱', '距離(公尺)', '預計時間']].dropna().head(3)
        return res

In [4]:
# 測試
info = dotenv.dotenv_values('.env')
apikey = info.get('googleAPI')
location = '24.171550, 120.645647'

# 連線資料庫、SQL搜尋、轉df
conn = sqlite3.connect('TW_cinema_info.db')
data_df = pd.read_sql_query("SELECT * FROM TW_cinema_info_latest", conn)
conn.close()

# 建立物件
finder = CinemaFinder(apikey, location)

# 過程操作
filtered_data = finder.MathJudge_dist(data_df)
result = finder.order_df_dist_google(filtered_data)